In [1]:
from re import finditer, search
import pandas as pd
import numpy as np

In [2]:
# création d'un dataFrame à partir du fichier csv transmis par Jessy
df = pd.read_csv('source/export_data.csv')

In [3]:
# renommage de la colonne Unnamed: 0 qui correspond à l'id de chaque profil
df.rename(columns={'Unnamed: 0': 'id_profil'}, inplace=True)

In [4]:
non_atomique_cols = ['positions','educations','skills','location']
atomique_cols = [col for col in df.columns if col not in non_atomique_cols]
df_profils =  df[atomique_cols]

df_profils['num-connections'].replace('0.0','0',inplace=True)
df_profils['num-connections'] = df_profils['num-connections'].str.replace('+','')
df_profils['num-connections'] = df_profils['num-connections'].astype('float').astype('int')

df_profils.to_csv('destination/profils.csv', sep=";", index=False)

C:\Users\elias\logiciels\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\elias\AppData\Local\Temp/ipykernel_58352/3015813109.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_profils['num-connections'] = df_profils['num-connections'].str.replace('+','')
C:\Users\elias\AppData\Local\Temp/ipykernel_58352/3015813109.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [5]:
# reg_sep : regex qui va permettre de séparer pour chaque profil les dictionnaires contenus dans les listes pour les colonnes
# positions et educations.
# reg_col : regex qui va permettre pour chaque dictionnaire de séparer chaque paire clé - valeur.

reg_sep = "{.+?}"
reg_sep_skill = "'.+?'"
reg_col = "'[^:,]+': .+?(,|})"

sep = lambda column,num_line : [unique.group() for unique in finditer(reg_sep,df[column][num_line])]
sep_skills = lambda num_line : [unique.group().replace("'",'') for unique in finditer(reg_sep_skill,df['skills'][num_line])]

sep_range = lambda col,i_line: range(len(sep(col,i_line)))
regs = lambda cols : ["'"+col+".*?': (('.+?')|(\".+?\")|(.+?))(,|})" for col in cols]
sub_vals = lambda col,regs_,def_val : [[df['id_profil'][i_line], i_sep+1] + [search(regs_[i_reg],sep(col,i_line)[i_sep]).group().split(': ')[1].replace("'",'').replace('"','').strip(',}') if search(regs_[i_reg],sep(col,i_line)[i_sep]) is not None else def_val[i_reg] for i_reg in range(len(regs_))] for i_line in range(df.shape[0]) for i_sep in sep_range(col,i_line)]

In [6]:
educations_cols = [i.group().split(': ')[0].replace("'",'') for i in finditer(reg_col,sep('educations',0)[0])]
regs_educations = regs(educations_cols)
educations_default = [None for i in range(len(educations_cols))]
sub_vals_educations = sub_vals('educations',regs_educations,educations_default)
educations_cols = ['id_profil','num_education'] + educations_cols

In [7]:
df_educations = pd.DataFrame(sub_vals_educations, columns = educations_cols)
df_educations.to_csv('destination/educations.csv', sep=";", index=False)

In [8]:
skills_cols = ['id_profil','skill']
sub_vals_skills = [[df['id_profil'][i_line], skill] for i_line in range(df.shape[0]) for skill in sep_skills(i_line)]

In [9]:
df_skills = pd.DataFrame(sub_vals_skills, columns = skills_cols)
df_skills.to_csv('destination/skills.csv', sep=";", index=False)

In [10]:
positions_cols = ["summary","title","start-date","end-date","is-current","company-name"]
regs_positions = regs(positions_cols)
positions_default = ['False' if col == 'is-current' else None for col in positions_cols]
sub_vals_positions = sub_vals('positions',regs_positions,positions_default)
positions_cols = ['id_profil','num_position'] + positions_cols

In [11]:
df_positions = pd.DataFrame(sub_vals_positions, columns = positions_cols)
df_positions.to_csv('destination/positions.csv', sep=";", index=False)

In [12]:
fileStates = open("source/states.txt", "r")
states_list = fileStates.readlines()
fileStates.close()
states = [state[:-2] for state in states_list]

In [13]:
nb_lines = range(df.shape[0])
sepLoc = lambda i_line : df.location[i_line].split(',')

repV2 = lambda s : s.replace(' Area','').replace('Ciudad de ','').replace(' y alrededores','').replace(' e Região','').replace('Région de','').replace(' County','').strip(' ') 
repV1 = lambda s : s.replace(' Area','').replace('Greater ','').replace(' Metro','')
repS = lambda s : s.replace(' Area','').strip(' ')
repC = lambda s : s.strip(' ').replace('Canada Area', 'Canada')

In [14]:
profil = lambda i_line : df['id_profil'][i_line]
city = lambda i_line : (repV2(sepLoc(i_line)[0]) if (len(sepLoc(i_line))>1) else (repV1(sepLoc(i_line)[0]) if ('Area' in sepLoc(i_line)[0]) else None))
state = lambda i_line : (sepLoc(i_line)[1].strip(' ') if (len(sepLoc(i_line))==3) else (repS(sepLoc(i_line)[-1]) if ((len(sepLoc(i_line))==2) and (('Columbia' in sepLoc(i_line)[-1]) or (np.sum([state not in (sepLoc(i_line)[-1]) for state in states])==len(states)-1))) else None))
country = lambda i_line : (repC(sepLoc(i_line)[-1]) if (('Area' not in repC(sepLoc(i_line)[-1])) and ('Columbia' not in sepLoc(i_line)[-1]) and np.sum([state not in (sepLoc(i_line)[-1]) for state in states])==len(states)) else "United States")

locations_cols = ['id_profil', 'city', 'state', 'country']
vals_locations = [[profil(i_line), city(i_line), state(i_line), country(i_line)] for i_line in nb_lines]

In [15]:
df_locations = pd.DataFrame(vals_locations, columns = locations_cols)
df_locations.to_csv('destination/locations.csv', sep=";", index=False)